In [ ]:
import torchvision
import numpy as np
import matplotlib
import wandb
from src import preprocess, activation_functions, initializers, layer, loss_functions, optimizers, FFNN, training, wandblogger, dropout, batchnorm, sweeping
import nbformat


In [ ]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data/fashion-mnist', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data/fashion-mnist', train=False, download=True)

In [ ]:
validation_percent = 0.2
one_hot_encoder = preprocess.OneHotEncoder()


train_size = int((1 - validation_percent) * len(train_dataset))
validation_size = len(train_dataset) - train_size

x_train = train_dataset.data[:train_size].reshape(train_size, -1).numpy()
targets_train = np.array(train_dataset.targets[:train_size])
y_train = one_hot_encoder.transform(targets_train)

x_valid = train_dataset.data[train_size:].reshape(validation_size, -1).numpy()
targets_valid = np.array(train_dataset.targets[train_size:])
y_valid = one_hot_encoder.transform(targets_valid)

x_test = test_dataset.data.reshape(len(test_dataset), -1).numpy()
targets_test = np.array(test_dataset.targets)
y_test = one_hot_encoder.transform(targets_test)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

In [ ]:
#find min and max value of y_train
print("y_train min:", np.min(y_train), "y_train max:", np.max(y_train))

In [ ]:
project = "Fashion-MNIST"
entity = "DTU-Deep-Learning-Project"

In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "optimizer": {"values": ["adam"]},
        "learning_rate": {"min": 0.00001, "max": 0.1},
        "l2_lambda": {"min": 0.0, "max": 0.0001},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 20},
        "normalize_data": {"values": [False, True]},
        "activation_function": {"values": ["ReLU()", "LeakyReLU()", "Tanh()"]},
        "kernel_initializer": {"values": ["HeInitializer()", "GlorotInitializer()"]},
        # 🔥 The important part:
        # Hidden layer configurations can vary
        "layer_sizes": {
            "values": [
                [784, 256],
                [784, 512],
                [784, 1024, 512],
                [784, 512, 256],
                #[784, 512, 512, 256, 128],
                #[784, 2048, 1024, 512, 256, 128, 64]
            ]
        },
        
        "use_batchnorm": {"values": [True, False]},
        "dropout_rate": {"min": 0.0, "max": 0.5},
    },
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity=entity,
    project=project)

import src.profiler as profiler

lambda_function = lambda: sweeping.train_sweep(entity, project, config=sweep_config, x_train=x_train, y_train=y_train, x_valid=x_valid, y_valid=y_valid)
#lambda_function = lambda: profiler.profile_sweep(entity, project, sweep_config, x_train, y_train, x_valid, y_valid)

wandb.agent(sweep_id, function=lambda_function, count=1)